## This notebook formats the data published by the Department of Fisheries and Oceans Canada (DFO)
### Note that this formatted data is not merged into the rest of the industry counts and is not used in the figures or tables.

The data from DFO consists of lice counts on salmon farms. The counts are reported by the companies. The data here was downloaded from [Industry sea lice counts at BC marine finfish aquaculture sites](https://open.canada.ca/data/en/dataset/3cafbe89-c98b-4b44-88f1-594e8d28838d) from the Government of Canada Open Government site on 17 March 2025.

We format the DFO count data to match that in *industry_farm_abundances.csv*. We replace the DFO field *Facility Reference Number* with *facility_id* to link to site descriptions in *industry_farm_details.csv*.

In [2]:
import pandas as pd
from pathlib import Path

### Path configuration variables

In [28]:
# DFO data path
dfo_farm_data_filepath = Path('DFO') / 'lice-count-dens-pou-2011-ongoing-rpt-pac-dfo-mpo-aquaculture-eng.csv'
# industry data path
industry_farm_details_filepath = Path('.') / 'industry_farm_details.csv'

# output for formatted data
dfo_formatted_farm_filepath = Path('.') / 'DFO_farm_abundance.csv'

# mapping file from DFO 'Facility Reference Number' to 'facility_id'
ref_to_id_map_filepath = Path('.') / 'DFO_facility_mapping.csv'